<a href="https://colab.research.google.com/github/OneFineStarstuff/OneFineStardust/blob/main/Federated_Learning_for_Decentralized_Data_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install syft

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import syft as sy

# Initialize PySyft hook
hook = sy.TorchHook(torch)

# Initialize workers
alice = sy.VirtualWorker(hook, id="alice")
bob = sy.VirtualWorker(hook, id="bob")

# Define a simple model
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc = nn.Linear(10, 2)

    def forward(self, x):
        return self.fc(x)

# Create a model
model = Net()

# Dummy data
data_alice = torch.randn(32, 10).send(alice)
target_alice = torch.randint(0, 2, (32,)).send(alice)
data_bob = torch.randn(32, 10).send(bob)
target_bob = torch.randint(0, 2, (32,)).send(bob)

# Optimizer
optimizer = optim.SGD(model.parameters(), lr=0.01)

# Federated learning training
for epoch in range(10):
    # Train on Alice's data
    model.send(alice)
    optimizer.zero_grad()
    output = model(data_alice)
    loss = nn.CrossEntropyLoss()(output, target_alice)
    loss.backward()
    optimizer.step()
    model.get()

    # Train on Bob's data
    model.send(bob)
    optimizer.zero_grad()
    output = model(data_bob)
    loss = nn.CrossEntropyLoss()(output, target_bob)
    loss.backward()
    optimizer.step()
    model.get()